<a href="https://colab.research.google.com/github/daniilprigozhin/NLRome_Align_Tree/blob/main/Copy_of_Soy_CladeFinder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This program takes a immunoprotein sequence for Soy, Maize, or Arabidopsis and returns the clade in which this protein lies in relation to known subspecies. We also return a Shannon Entropy model for analyzing highly variable portions of the protein, which can be viewed on Chimera. Please use Chrome for this process to ensure results.

In [ ]:
#@title Install Non-Python Dependencies
#@markdown This block may take 4-5 minutes. If it says your session crashed, do not be alarmed and continue running the next code block. This is a bug from importing one of the necessary packages onto Colab.

!pip install -q condacolab &> /dev/null
import condacolab
condacolab.install()
!conda install -c bioconda hmmer &> /dev/null
!conda install -c bioconda mafft &> /dev/null
!pip install Bio &> /dev/null
from collections import defaultdict
from Bio import SearchIO
!conda install -c bioconda epa-ng &> /dev/null
!conda install -c bioconda gappa &> /dev/null
from google.colab import files
!pip install toytree &> /dev/null

from Bio import AlignIO

In [ ]:
#@title Install Python Dependencies
!pip install -q gwpy &> /dev/null
!conda install -c bioconda hmmer &> /dev/null
!conda install -c bioconda mafft &> /dev/null
import math
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import toytree       # a tree plotting library
import toyplot       # a general plotting library

In [ ]:
#@title Select Species
Species = "Arabidopsis" #@param ["Soy", "Arabidopsis", "Maize"]

%cd /content
!rm -rf NLRome_Align_Tree
!git clone https://github.com/daniilprigozhin/NLRome_Align_Tree &> /dev/null
 
%cd NLRome_Align_Tree/
if Species == 'Soy':
  %cd Soy_NLRome
  common = pd.read_csv("Soy_NLRome_GeneTable.txt", sep='\t')
elif Species == "Arabidopsis":
  %cd Atha_NLRome
  common = pd.read_csv("Atha_NLRome_GeneTable.txt", sep = '\t')
else:
  %cd Maize_NLRome
  common = pd.read_csv("Maize_NLRome_GeneTable.txt", sep = '\t')


clades = common[["Clade", "File"]].groupby("Clade").first()
!unzip All_Clades.zip

In [ ]:
# Code used from https://github.com/sokrypton/ColabFold
import re
import hashlib

def add_hash(x,y):
  return x+"_"+hashlib.sha1(y.encode()).hexdigest()[:5]

query_sequence = 'MGSAMSLSCSKRKATSQDLDSESCKRRKTCSTNDAENCRFIPDESSWSLCANRVISVALTKFRFQQDNQESNSSSLSLPSPATSVSRNWKHDVFPSFHGADVRRTFLSHILESFRRKGIDTFIDNNIERSKSIGPELKEAIKGSKIAIVLLSRKYASSSWCLDELAEIMICREVLGQIVMTIFYEVDPTDIKKQTGEFGKAFTKTCRGKPKEQVERWRKALEDVATIAGYHSHKWCDEAEMIEKISTDVSNMLDLSIPSKDFDDFVGMAAHMEMTEQLLRLDLDEVRMIGIWGPPGIGKTTIAACMFDRFSSRFPFAAIMTDIRECYPRLCLNERNAQLKLQEQMLSQIFNQKDTMISHLGVAPERLKDKKVFLVLDEVGHLGQLDALAKETRWFGPGSRIIITTEDLGVLKAHGINHVYKVKSPSNDEAFQIFCMNAFGQKQPCEGFWNLAWEVTCLAGKLPLGLKVLGSALRGMSKPEWERTLPRLKTSLDGNIGSIIQFSFDALCDEDKYLFLYIACLFNNESTTKVEEVLANKFLDVGQGIHVLAQKSLISFEGEEIQMHTLLVQFGRETSRKQFVHHRYTKHQLLVGERDICEVLNDDTIDSRCFIGINLDLSKNEERWNISEKALERMHDFQFVRIGAFYQRKRLSLALQDLIYHSPKLRSLKWYGYQNICLPSTFNPEFLVELDMSFSKLWNLWEGTKQLRNLKWMDLSYSSYLKELPNLSTATNLEELRLSNCSSLVELPSFGNATKLEKLDLENCRSLVKLPAIENATKLRKLKLEDCSSLIELPLSIGTATNLKKLDMNGCSSLVRLPSSIGDMTSLEGFDLSNCSNLVELPSSIGNLRKLALLLMRGCSKLETLPTNINLISLRILDLTDCSRLKSFPEISTHIDSLYLIGTAIKEVPLSIMSWSPLADFQISYFESLKEFPHAFDIITKLQLSKDIQEVPPWVKRMSRLRDLRLNNCNNLVSLPQLPDSLAYLYADNCKSLERLDCCFNNPEISLYFPNCFKLNQEARDLIMHTSTRNFAMLPGTQVPACFNHRATSGDTLKIKLKESPLPTTLRFKACIMLVKGYKEMRYDKIYVGTVIKDEQNDLKVLCTPSSCYIYPVLTEHIYTFELELKEVTSTELVFEFKSFRSNWKIGECGILQR' #@param {type:"string"}
# remove whitespaces
query_sequence = "".join(query_sequence.split())
query_sequence = re.sub(r'[^a-zA-Z]','', query_sequence).upper()

jobname = 'RPP1_NdA' #@param {type:"string"}
# remove whitespaces
jobname = "".join(jobname.split())
jobname = re.sub(r'\W+', '', jobname)
jobname_unhashed = jobname
jobname = add_hash(jobname, query_sequence)

%mkdir {jobname}
%cd {jobname}


with open(f"{jobname}.fasta", "w") as text_file:
    text_file.write(">" + jobname_unhashed + "\n%s" % query_sequence)

In [ ]:
#@title Create an Annotated Tree with your protein included
!hmmsearch --domtblout {jobname}.tbl.out ../All_Clades.hmm {jobname}.fasta &> /dev/null

filename = jobname + ".tbl.out"

attribs = ['bias', 'bitscore',  'evalue']

hits = defaultdict(list)
names = []

with open(filename) as handle:
    for queryresult in SearchIO.parse(handle, 'hmmsearch3-domtab'):
      #print(queryresult.id)
      #print(queryresult.accession)
      #print(queryresult.description)
      for hit in queryresult.hsps:
        names.append(queryresult.id)
        for attrib in attribs:
          hits[attrib].append(getattr(hit, attrib))

final = pd.DataFrame.from_dict(hits)
final["Clade"] = names
final.sort_values("bitscore", ascending = False)

#common = pd.read_csv("Soy_NLRome_GeneTable.txt", sep='\t')

final_hv = final.set_index("Clade").join(common[["Clade", "HV", "File"]].groupby("Clade").agg(max), how = 'left', rsuffix = "_other").sort_values("bitscore", ascending = False)
best_afa_file = final_hv.iloc[0,:]["File"]
best_afa = "../" + best_afa_file

!mafft --add {jobname}.fasta --keeplength {best_afa} > {jobname}.updated.afa

clade_oi = common[common["File"] == best_afa_file].iloc[0,:]["Clade"]
tree_oi = best_afa[:best_afa.find("Int")] + "RAxML_bestTree." + clade_oi + ".Raxml.out"
raxml_info_oi = best_afa[:best_afa.find("Int")] + "RAxML_info." + clade_oi + ".Raxml.out"

!epa-ng --split {best_afa} {jobname}.updated.afa --tree {tree_oi} --model {raxml_info_oi}
!cut -f1 -d ' ' {best_afa} > best_afa_f.afa
!epa-ng --ref-msa best_afa_f.afa --tree {tree_oi} -q query.fasta --model {raxml_info_oi} --redo

!gappa examine graft --jplace-path ./ &> /dev/null

tree_created = True

is_hv = final_hv.iloc[0,:]["HV"] == 1

In [ ]:
#@title View tree
directory = os.getcwd()
file_name = directory + "/epa_result.newick" 
tre = toytree.tree(file_name)
colorlist = ["#d6557c" if jobname_unhashed in tip else "#5384a3" for tip in tre.get_tip_labels()]

tre.draw(tip_labels_align=False,layout='d',tip_labels_colors=colorlist,tip_labels_style={"font-size": "8px"});

In [ ]:
#@title This block will tell you if your sequence falls within a highly variable clade.
if is_hv:
  print("Your protein belongs to a highly variable clade.")
else:
  print("Your protein is not in a highly variable clade.")


In [ ]:
#@title Above ends the process of tree creation. Here we create a Shannon Entropy model (receommended for proteins in highly variable clades), and print the graphed entropy results.
def entropy(string):
    "Calculates the Shannon entropy of a string"
    string = string.replace("-","")
    # get probability of chars in string
    prob = [ float(string.count(c)) / len(string) for c in dict.fromkeys(list("ARNDCQEGHILKMFPSTWYV")) ]

    # calculate the entropy
    entropy = - sum([ p * math.log(p) / math.log(2.0) for p in prob if p > 0])

    return entropy

gene = query_sequence
cn = jobname_unhashed
file_oi = jobname + ".updated.afa"

align = AlignIO.read(file_oi, "fasta")

length = len(align[1,:])
entropies = []
for i in range(length):
  col = align[:,i]
  if col[len(align) - 1] == "-":
    continue
  entropies.append(entropy(col))

f = open(jobname+"_Chimera_Entropy.txt", "w")
f.write("attribute: shannonEntropy\n")
f.write("match mode: 1-to-1\n")
f.write("recipient: residues\n")
for i in range(1,len(entropies)+1):
  f.write("\t:")
  f.write(str(i))
  f.write("\t")
  f.write(str(entropies[i-1]))
  f.write("\n")

ylim = max(max(entropies), 2.5)

plt.ylim(0,ylim);
plt.plot(entropies);
plt.xlabel("Residue Number");
plt.ylabel("Entropy (bits)");
plt.title("Shannon Entropy of "+jobname_unhashed);
plt.savefig(jobname + "_Entropy_MaskedNG");

shannon_created = True

In [ ]:
#@title Download Results
#@markdown Our assembled zip file contains the tree created in Newick format and an updated aligned fasta format of your sequence against all other proteins in your species. \
#@markdown \
#@markdown If a Shannon Entropy model was generated, it also includes the Chimera-formatted entropy mapping, as well as the graph generated above of entropy per residue. \
#@markdown \
#@markdown If you are having issues downloading the result, try disabling your adblocker and run this cell again. If that fails click on the folder icon to the left, navigate to /content, and manually download {jobname}_results.zip.

new_folder = jobname_unhashed + "_results"
!mkdir {new_folder}
!mv {jobname}.updated.afa {new_folder}
!mv epa_result.newick {new_folder}
if shannon_created:
  %mv {jobname}_Chimera_Entropy.txt {new_folder}
  %mv {jobname}_Entropy_MaskedNG.png {new_folder}

!mv {new_folder} ../../../
%cd /content
!zip -r {new_folder}.zip {new_folder}
res = new_folder + ".zip"
files.download(res)

**Sources:**